# Callback回调机制

[![下载Notebook](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_notebook.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_callback.ipynb)&emsp;
[![下载样例代码](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_download_code.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_callback.py)&emsp;
[![查看源文件](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/tutorials-develop/tutorials/source_zh_cn/intermediate/callback.ipynb)

在深度学习训练过程中，为及时掌握网络模型的训练状态、实时观察网络模型各参数的变化情况和实现训练过程中用户自定义的一些操作，MindSpore提供了Callback回调机制。

Callback回调机制一般用在网络模型训练过程`Model.train`中，MindSpore会按照Callback列表`callbacks`顺序执行回调函数，用户可以通过配置不同的回调函数来实现不同功能。

## 常用的内置回调函数

下面以基于MNIST数据集训练LeNet-5网络模型为例，介绍几种常用的MindSpore内置回调函数，更多内置回调函数信息及使用方式请参考[API文档](https://www.mindspore.cn/docs/api/zh-CN/master/api_python/mindspore.train.html#mindspore-train-callback)。首先需要下载并处理MNIST数据，构建LeNet-5网络模型，示例代码如下：

In [1]:
import mindspore.nn as nn
from mindspore.train import Model
from mindvision.classification.dataset import Mnist
from mindvision.classification.models import lenet

# 使用MindSpore Vision套件提供的Mnist接口下载并处理MNIST数据集
download_train = Mnist(path="./mnist", split="train", batch_size=32, repeat_num=1, shuffle=True, resize=32, download=True)
download_eval = Mnist(path="./mnist", split="test", batch_size=32, resize=32, download=True)
dataset_train = download_train.run()
dataset_eval = download_eval.run()

# 使用MindSpore Vision套件提供的lenet接口实例化LeNet-5网络模型
network = lenet(num_classes=10, pretrained=False)

# 定义损失函数
net_loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')

# 定义优化器
net_opt = nn.Momentum(network.trainable_params(), learning_rate=0.01, momentum=0.9)

# 定义网络模型
model = Model(network, loss_fn=net_loss, optimizer=net_opt, metrics={"Accuracy": nn.Accuracy()})

## ModelCheckpoint

为了保存训练后的网络模型和参数，方便进行再推理或再训练，MindSpore提供了[ModelCheckpoint](https://mindspore.cn/docs/api/zh-CN/master/api_python/mindspore.train.html?highlight=modelcheckpoint#mindspore.train.callback.ModelCheckpoint)接口，一般与配置保存信息接口[CheckpointConfig](https://mindspore.cn/docs/api/zh-CN/master/api_python/mindspore.train.html?highlight=modelcheckpoint#mindspore.train.callback.CheckpointConfig)配合使用，下面我们通过一段示例代码来说明如何保存训练后的网络模型和参数。

In [3]:
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig

# 设置保存模型的配置信息
config_ck = CheckpointConfig(save_checkpoint_steps=1875, keep_checkpoint_max=10)
# 实例化保存模型回调接口，定义保存路径和前缀名
ckpoint = ModelCheckpoint(prefix="lenet", directory="./lenet", config=config_ck)

# 开始训练，加载保存模型和参数回调函数
model.train(1, dataset_train, callbacks=[ckpoint], dataset_sink_mode=False)

上面代码运行后，生成的Checkpoint文件目录结构如下：

```text
./lenet/
├── lenet-1_1875.ckpt # 保存参数文件
└── lenet-graph.meta # 编译后的计算图
```

## LossMonitor

为了监控训练过程中的损失函数值Loss变化情况，观察训练过程中每个epoch、每个step的运行时间，MindSpore Vision提供了`LossMonitor`接口，下面我们通过示例代码说明：

In [4]:
from mindvision.engine.callback import LossMonitor

# 开始训练，加载保存模型和参数回调函数，LossMonitor的入参0.01为学习率，200为步长
model.train(1, dataset_train, callbacks=[LossMonitor(0.01, 200)], dataset_sink_mode=False)

Epoch:[  0/  1],                     step:[  200/ 1875],                    loss:[0.013/0.045],                    time:6.496,                     lr:0.01000.
Epoch:[  0/  1],                     step:[  400/ 1875],                    loss:[0.010/0.052],                    time:6.119,                     lr:0.01000.
Epoch:[  0/  1],                     step:[  600/ 1875],                    loss:[0.188/0.050],                    time:5.910,                     lr:0.01000.
Epoch:[  0/  1],                     step:[  800/ 1875],                    loss:[0.027/0.052],                    time:7.077,                     lr:0.01000.
Epoch:[  0/  1],                     step:[ 1000/ 1875],                    loss:[0.188/0.050],                    time:5.287,                     lr:0.01000.
Epoch:[  0/  1],                     step:[ 1200/ 1875],                    loss:[0.014/0.049],                    time:6.299,                     lr:0.01000.
Epoch:[  0/  1],                     step:[ 14

从上面的打印结果可以看出，由于步长设置的是200，所以每200个step会打印一条，loss值会波动，但总体来说loss值会逐步减小，精度逐步提高。每个人运行的loss值有一定随机性，不一定完全相同。

## ValAccMonitor

为了在训练过程中保存精度最优的网络模型和参数，需要边训练边验证，MindSpore Vision提供了`ValAccMonitor`接口，下面我们通过一段示例来介绍：

In [ ]:
from mindvision.engine.callback import ValAccMonitor

# 开始训练，加载保存模型和参数回调函数
model.train(1, dataset_train, callbacks=[ValAccMonitor(model, dataset_eval, num_epochs=1, dataset_sink_mode=False)], dataset_sink_mode=False)

上面代码执行后，精度最优的网络模型和参数会被保存在当前目录下，文件名为"best.ckpt"，打印结果如下：

```text
--------------------
Epoch: [  1 /   1],                   Train Loss: [0.135],                   Accuracy:  0.988.
================================================================================
End of validation the best Accuracy is:  0.988,               save the best ckpt file in ./best.ckpt
```


## 自定义回调函数

MindSpore不仅有功能强大的内置回调函数，当用户有自己的特殊需求时，还可以基于`Callback`基类自定义回调函数。

回调机制可以把训练过程中的重要信息记录下来，通过把一个字典类型变量`RunContext.original_args()`传递给Callback对象，使得用户可以在各个自定义的Callback中获取到相关属性，执行自定义操作，也可以自定义其他变量传递给`RunContext.original_args()`对象。

`RunContext.original_args()`中的常用属性有：

- loss_fn：损失函数
- optimizer：优化器
- train_dataset：训练的数据集
- epoch_num：训练的epoch的数量
- batch_num：一个epoch中step的数量
- train_network：训练的网络
- cur_epoch_num：当前的epoch数
- cur_step_num：当前的step数
- parallel_mode：并行模式
- list_callback：所有的Callback函数
- net_outputs：网络的输出结果

通过下面两个场景，我们可以增加对自定义Callback回调函数功能的了解。

1. 实现在规定时间内终止训练。用户可以设定时间阈值，当训练时间达到这个阈值后就终止训练过程。

In [5]:
import time
from mindspore.train.callback import Callback

# 自定义回调类
class StopAtTime(Callback):
    # 定义初始化过程
    def __init__(self, run_time):
        super(StopAtTime, self).__init__()
        # 定义执行时间
        self.run_time = run_time

    # 开始训练时的操作
    def begin(self, run_context):
        cb_params = run_context.original_args()
        # 获取当前时间戳作为开始训练时间
        cb_params.init_time = time.time()
        print("Begin training, time is:", cb_params.init_time)

    # 每个step结束后执行的操作
    def step_end(self, run_context):
        cb_params = run_context.original_args()
        # 获取epoch值
        epoch_num = cb_params.cur_epoch_num
        # 获取step值
        step_num = cb_params.cur_step_num
        # 获取损失值loss
        loss = cb_params.net_outputs
        # 获取当前时间戳
        cur_time = time.time()

        if (cur_time - cb_params.init_time) > self.run_time:
             # 当运行的时间大于设定的阈值时，打印信息
            print("End training, time is:", cur_time, "   epoch:", epoch_num, " step:", step_num, " loss:", loss)
            # 停止训练
            run_context.request_stop()

# 开始训练
model.train(1, dataset_train, callbacks=[LossMonitor(0.01, 200), StopAtTime(10)], dataset_sink_mode=False)

Begin training, time is: 1646988005.0114133
Epoch:[  0/  1],                     step:[  200/ 1875],                    loss:[0.000/0.030],                    time:6.537,                     lr:0.01000.
Epoch:[  0/  1],                     step:[  400/ 1875],                    loss:[0.004/0.031],                    time:6.731,                     lr:0.01000.
Epoch:[  0/  1],                     step:[  600/ 1875],                    loss:[0.007/0.033],                    time:6.998,                     lr:0.01000.
Epoch:[  0/  1],                     step:[  800/ 1875],                    loss:[0.011/0.034],                    time:8.863,                     lr:0.01000.
Epoch:[  0/  1],                     step:[ 1000/ 1875],                    loss:[0.018/0.034],                    time:7.650,                     lr:0.01000.
Epoch:[  0/  1],                     step:[ 1200/ 1875],                    loss:[0.011/0.035],                    time:7.834,                     lr:0.01000.
En

从上面的打印结果可以看出，当执行到第1263个step时运行时间到达了阈值并结束了训练。

2. 实现当loss小于设定的阈值时，保存ckpt文件。示例代码如下：

In [6]:
from mindspore import save_checkpoint
from mindspore.train.callback import Callback

# 定义保存ckpt文件的回调接口
class SaveCallback(Callback):
    # 定义初始化过程
    def __init__(self, loss):
        super(SaveCallback, self).__init__()
        # 定义损失值阈值
        self.loss = loss

    # 定义step结束时的执行操作
    def step_end(self, run_context):
        cb_params = run_context.original_args()
        # 获取当前损失值
        cur_loss = cb_params.net_outputs.asnumpy()
        # 如果当前损失值小于设定的阈值就停止训练
        if cur_loss < self.loss:
            # 自定义保存文件名
            file_name = str(cb_params.cur_epoch_num) + "_" + str(cb_params.cur_step_num) + ".ckpt"
            # 保存网络模型
            save_checkpoint(save_obj=cb_params.train_network, ckpt_file_name=file_name)
            print("Saved checkpoint, loss:{:8.7f}, current step num:{:4}.".format(cur_loss, cb_params.cur_step_num))

# 开始训练
model.train(1, dataset_train, callbacks=[SaveCallback(5e-5)], dataset_sink_mode=False)

Saved checkpoint, loss:0.0000225, current step num:1085.
Saved checkpoint, loss:0.0000291, current step num:1181.
Saved checkpoint, loss:0.0000346, current step num:1616.


保存目录结构如下：

```text
./
├── 1_1085.ckpt
├── 1_1181.ckpt
├── 1_1616.ckpt
```